In [ ]:

from preproc_tabular import get_tabular
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
desc_path= 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni.pt'
desc_tot_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni_tot.pt'

data_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_week.csv'
data_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_month.csv'
data_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_season.csv'

noneg_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_week.csv'
noneg_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_month.csv'
noneg_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_season.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

modello = sp.create_model(1, 1).to(device)
batch_size = 64
epochs = 20
criterion = nn.MSELoss()


optimizer = AdamW(
    [
        {"params": modello.image.parameters(), "lr": 1e-3},
        {"params": modello.tabular.parameters()},
        {"params": modello.description.parameters(), "lr": 1e-3},
        {"params": modello.final.parameters()}
    ],
    lr=1e-2,
)
early_stop = sp.EarlyStopping(7, 0.001)

data, references, descriptions, target = get_tabular(aggregated_path, desc_path)

train, val = getDataset(references, data, descriptions, target, img_path, batch_size, 1)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
min_mae = 100000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2, bias = sp.validation_loop(val, modello, criterion, device)
    if (min_mae > mae):
        modello_final = modello
        min_mae = mae
    early_stp = early_stop(mse)
    
    if early_stp:
        print('Early Stop attivato.')
        break
    
    
print("Done!")

folder = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\season_negozi'
if not os.path.isdir(folder):
    os.makedirs(folder)
model_sc = torch.jit.script(modello_final)
model_sc.save(f'{folder}\\weights.pt')